In [1]:
import mysql.connector as mariadb
from random import randint

In [2]:

watering_duration = randint(300, 1800)
# seconds. Randomly calculate time between 5 and 30 mins, to test duration options

time_since_rules_check = 24 # if latest rules greater than this value, execute exception process
exception_hours_between_watering = 70

# Set db variables

db_host = 'hda.amahi.net'
db_host_port = '3306'
db_user = 'rpi'
db_pass = 'warm_me'
db = 'soil'

# Initialise variables

hold_watering = True
hours_since_last_rules = 999
watered = False
duration = 0.0

In [3]:
query_rules = """
SELECT
hold_watering,
TIMESTAMPDIFF(hour,ts,NOW())
FROM watering
order by ts
"""

con = mariadb.connect(host=db_host, port=db_host_port,
                      user=db_user, password=db_pass, database=db)
cur = con.cursor()

cur.execute(query_rules)

for row in cur:
    hold_watering = bool(row[0])
    hours_since_last_rules = row[1]

In [4]:
hold_watering, hours_since_last_rules

(True, 16)

In [5]:

query_exception = """
SELECT
TIMESTAMPDIFF(hour, ts, NOW()),
watered
FROM watering_records
WHERE watered in('1', 'True')
ORDER BY ts ASC
"""

con = mariadb.connect(host=db_host, port=db_host_port,
user=db_user, password=db_pass, database=db)

cur = con.cursor()

cur.execute(query_exception)

hours_since_last_water = 999

for row in cur:
    hours_since_last_water = row[0]

if hours_since_last_rules > time_since_rules_check:
    if hours_since_last_water > exception_hours_between_watering:
        hold_watering = False
    else:
        hold_watering = True



In [6]:
hours_since_last_water


382

In [7]:
hold_watering

True